## Problem: 

The data we will work with comes from 5 sensors placed in an office collecting data on light, temperature, humidity and CO2 measurements. Every minute the sensor takes a reading and the occupancy of the room is determined. The data were collected with the intention of determining the preferred environmental conditions for office workers.

Data source:  
*Accurate occupancy detection of an office room from light, temperature, humidity and CO2 measurements using statistical learning models. Luis M. Candanedo, Varonique Feldheim. Energy and Buildings. Volume 112, 15 January 2016, Pages 28-39.*

<img src="office.jpeg">

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.neighbors import LocalOutlierFactor
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Reading the data
df = pd.read_csv('occupancy.csv')
df.head(10)

In [ ]:
# Dropping the non-required columns
df = df.drop(['date','ID','Occupancy'],axis=1)

In [ ]:
df.info()

### Basic Technique: Z-scores

In [ ]:
# Z-score function for detecting outliers
'''
	We have kept the threshold for outliers to be 3 i.e. 
	if the z_score of a values is greater than 3 or less than -3 
	then it shall be classified as an outlier
'''	
def outliers_z_score(data,threshold=3):
	mean_data = np.mean(data)
	stdev_data = np.std(data)
	z_scores = [(y - mean_data) / stdev_data for y in data]
	return np.where(np.abs(z_scores) > threshold)


In [ ]:
# Applying the Z-score outlier detection technique on the data
for x in df:
    if x!='Occupancy':	
        result = outliers_z_score(df[x])

        # Replacing the detected outliers with 'ANOMALY'
        for y in result:
            df[x][y]='ANOMALY'

In [ ]:
# inspect data
df.head(20)

### Advanced Technique: Local Outlier Factors (LOF)

In [ ]:
# Reading the data
df = pd.read_csv('occupancy.csv')
df.head(10)

In [ ]:
# Dropping the non-required columns
df = df.drop(['date','ID','Occupancy'],axis=1)

In [ ]:
# Applying the Local Outlier Factor method to detect outliers
lof = LocalOutlierFactor()
result = lof.fit_predict(df)


In [ ]:
# Adding the result column to the data
'''
	In the result column named as 'outlier_detected':
		1 is for the records which are clean 
		-1 is for the records which are detected as outliers
'''
df['outlier_detected']=result


In [ ]:
# inspect data
df.head(20)